In [1]:
import pandas as pd
from preprocess import Audio_Processor
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

SR = 44100
blocksize = 88200
overlap = 44100

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
audio_dir = '../ESC-50/audio/'
path_to_db='../ESC-50/'
ps = Audio_Processor(audio_dir)

# Load Dataset
Here we load the csv that describes each file in the dataset. We add a high level category that is defined in the ESC-50 documentation. This we realize is anthetical to true training, it is a stopgap for when we use NLP to classify tags into these categories.

In [5]:
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']

In [6]:
mapping = {'dog': 0,'rooster': 0,'pig': 0,'cow': 0,'frog': 0,'cat': 0,'hen': 0,
            'insects': 0,'sheep': 0,'crow': 0,'rain': 1,'sea_waves': 1,'crackling_fire': 1,
            'crickets': 0,'chirping_birds': 0,'water_drops': 1,'wind': 1,'pouring_water': 1,
            'toilet_flush': 1,'thunderstorm': 1,'crying_baby': 0,'sneezing': 0,'clapping': 0,
            'breathing': 0,'coughing': 0,'footsteps': 1,'laughing': 0,'brushing_teeth': 1,
            'snoring': 0,'drinking_sipping': 1,'door_wood_knock': 1,'mouse_click': 1,
            'keyboard_typing': 1,'door_wood_creaks': 1,'can_opening': 1,'washing_machine': 1,
            'vacuum_cleaner': 1,'clock_alarm': 1,'clock_tick': 1,'glass_breaking':1,'helicopter': 1,
            'chainsaw': 1,'siren': 1,'car_horn': 1,'engine': 1,'train': 1,'church_bells': 1,
            'airplane': 1,'fireworks': 1,'hand_saw': 1,
            }

In [7]:
dataset['h_category'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_category'] = mapping[row['category']]

In [8]:
dataset.head()

,filename,fold,target,category,esc10,src_file,take,h_category
0,1-100032-A-0.wav,1,0,dog,True,100032,A,0
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,0
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,1
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,1
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,1


## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [9]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        fld=range(2,6), 
                        blocksize=blocksize, 
                        overlap=overlap)

In [10]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,l_target,h_target
count,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,...,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000,6058.000000
mean,0.274425,0.278048,0.196117,0.174222,0.157807,0.148653,0.138702,0.139089,0.129341,0.126932,...,-0.000091,0.000016,-0.000070,0.000057,-0.000020,0.000020,-0.000041,-0.000003,0.623968,24.517993
std,0.151244,0.153651,0.086790,0.075001,0.065857,0.060723,0.055634,0.057795,0.051140,0.050487,...,0.001409,0.001365,0.001341,0.001381,0.001393,0.001380,0.001329,0.001391,0.484428,14.509881
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.071744,-0.071744,-0.071744,-0.019164,-0.019158,-0.060990,-0.060990,-0.060990,0.000000,0.000000
25%,0.152635,0.148390,0.136185,0.122215,0.114203,0.109709,0.102971,0.099801,0.096195,0.094515,...,-0.000131,-0.000038,-0.000118,-0.000044,-0.000108,-0.000052,-0.000098,-0.000055,0.000000,12.000000
50%,0.257627,0.270524,0.195047,0.173687,0.156311,0.147308,0.139855,0.138163,0.129147,0.126159,...,-0.000032,0.000028,-0.000026,0.000018,-0.000022,0.000011,-0.000015,0.000004,1.000000,24.000000
75%,0.376937,0.386941,0.251302,0.221092,0.197469,0.184328,0.171041,0.173471,0.158836,0.156608,...,0.000033,0.000125,0.000033,0.000107,0.000037,0.000092,0.000043,0.000081,1.000000,37.000000
max,0.782315,0.806713,0.617982,0.542252,0.532506,0.473240,0.473139,0.455183,0.469797,0.402358,...,0.012951,0.008564,0.016074,0.071744,0.071744,0.071744,0.071744,0.071744,1.000000,49.000000


In [11]:
X = df.drop(['l_target', 'h_target'], axis=1)
y = df['l_target']

## High-Level Shallow Nets
Train binary shallow nets for high level categories(animals, natural, human, domestic, urban)

In [12]:
clas = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=24, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
clas.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=24, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# Animal Sounds

In [14]:
a_data = X[y == 0]
a_targ = df['h_target'][y == 0]

In [15]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

def gpu_mfcc_deep_net_a():
    # Create Model
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(114,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))    
    model.add(Dense(19, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.summary()

    
    return model

In [16]:
anim = KerasClassifier(build_fn=gpu_mfcc_deep_net_a, 
                       epochs=500, 
                       batch_size=128, 
                       validation_split=0.15)
history = anim.fit(a_data, a_targ)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               11500     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
__________

1936/1936 [==============================] - 0s 63us/step - loss: 2.0678 - acc: 0.2774 - val_loss: 3.5494 - val_acc: 0.1140
Epoch 45/500
1936/1936 [==============================] - 0s 60us/step - loss: 2.0065 - acc: 0.2960 - val_loss: 3.5545 - val_acc: 0.1374
Epoch 46/500
1936/1936 [==============================] - 0s 60us/step - loss: 2.0166 - acc: 0.2986 - val_loss: 3.6165 - val_acc: 0.1345
Epoch 47/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.9755 - acc: 0.2903 - val_loss: 3.5159 - val_acc: 0.1111
Epoch 48/500
1936/1936 [==============================] - 0s 62us/step - loss: 1.9494 - acc: 0.3084 - val_loss: 3.6647 - val_acc: 0.1257
Epoch 49/500
1936/1936 [==============================] - 0s 60us/step - loss: 2.0078 - acc: 0.3120 - val_loss: 3.4463 - val_acc: 0.1871
Epoch 50/500
1936/1936 [==============================] - 0s 58us/step - loss: 1.9387 - acc: 0.3161 - val_loss: 3.4130 - val_acc: 0.1404
Epoch 51/500
1936/1936 [==============================

Epoch 104/500
1936/1936 [==============================] - 0s 58us/step - loss: 1.3373 - acc: 0.5150 - val_loss: 4.4602 - val_acc: 0.2135
Epoch 105/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.3584 - acc: 0.5041 - val_loss: 4.5624 - val_acc: 0.1930
Epoch 106/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.3010 - acc: 0.5088 - val_loss: 4.7952 - val_acc: 0.2018
Epoch 107/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.3808 - acc: 0.5031 - val_loss: 4.1975 - val_acc: 0.2164
Epoch 108/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.3511 - acc: 0.5083 - val_loss: 4.6050 - val_acc: 0.1842
Epoch 109/500
1936/1936 [==============================] - 0s 61us/step - loss: 1.2910 - acc: 0.5284 - val_loss: 4.5579 - val_acc: 0.2164
Epoch 110/500
1936/1936 [==============================] - 0s 60us/step - loss: 1.3141 - acc: 0.5212 - val_loss: 4.5103 - val_acc: 0.2135
Epoch 111/500
1936/1936 [=========

1936/1936 [==============================] - 0s 57us/step - loss: 1.0475 - acc: 0.6136 - val_loss: 5.6641 - val_acc: 0.2456
Epoch 164/500
1936/1936 [==============================] - 0s 61us/step - loss: 1.0237 - acc: 0.6317 - val_loss: 5.5971 - val_acc: 0.2310
Epoch 165/500
1936/1936 [==============================] - 0s 63us/step - loss: 1.0178 - acc: 0.6379 - val_loss: 5.3622 - val_acc: 0.2485
Epoch 166/500
1936/1936 [==============================] - 0s 58us/step - loss: 1.0136 - acc: 0.6235 - val_loss: 5.2608 - val_acc: 0.2427
Epoch 167/500
1936/1936 [==============================] - 0s 56us/step - loss: 1.1192 - acc: 0.6080 - val_loss: 5.4059 - val_acc: 0.2281
Epoch 168/500
1936/1936 [==============================] - 0s 57us/step - loss: 1.1238 - acc: 0.6126 - val_loss: 5.3465 - val_acc: 0.2222
Epoch 169/500
1936/1936 [==============================] - 0s 61us/step - loss: 1.0750 - acc: 0.6193 - val_loss: 4.8644 - val_acc: 0.2544
Epoch 170/500
1936/1936 [=======================

1936/1936 [==============================] - 0s 60us/step - loss: 0.8213 - acc: 0.7009 - val_loss: 5.8490 - val_acc: 0.2749
Epoch 223/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.7822 - acc: 0.7040 - val_loss: 6.2785 - val_acc: 0.2456
Epoch 224/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.8091 - acc: 0.7252 - val_loss: 6.1399 - val_acc: 0.2661
Epoch 225/500
1936/1936 [==============================] - 0s 58us/step - loss: 0.8310 - acc: 0.7045 - val_loss: 5.8968 - val_acc: 0.2310
Epoch 226/500
1936/1936 [==============================] - 0s 58us/step - loss: 0.8131 - acc: 0.7025 - val_loss: 5.8971 - val_acc: 0.2544
Epoch 227/500
1936/1936 [==============================] - 0s 58us/step - loss: 0.8085 - acc: 0.7128 - val_loss: 5.7597 - val_acc: 0.2661
Epoch 228/500
1936/1936 [==============================] - 0s 55us/step - loss: 0.8000 - acc: 0.7144 - val_loss: 5.5581 - val_acc: 0.2749
Epoch 229/500
1936/1936 [=======================

1936/1936 [==============================] - 0s 57us/step - loss: 0.6833 - acc: 0.7608 - val_loss: 6.0648 - val_acc: 0.2778
Epoch 282/500
1936/1936 [==============================] - 0s 56us/step - loss: 0.6473 - acc: 0.7681 - val_loss: 6.0660 - val_acc: 0.2690
Epoch 283/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.6828 - acc: 0.7603 - val_loss: 6.2233 - val_acc: 0.2602
Epoch 284/500
1936/1936 [==============================] - 0s 61us/step - loss: 0.6464 - acc: 0.7588 - val_loss: 6.1325 - val_acc: 0.2778
Epoch 285/500
1936/1936 [==============================] - 0s 59us/step - loss: 0.6478 - acc: 0.7665 - val_loss: 5.9591 - val_acc: 0.2836
Epoch 286/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.6433 - acc: 0.7774 - val_loss: 6.2851 - val_acc: 0.2836
Epoch 287/500
1936/1936 [==============================] - 0s 59us/step - loss: 0.6878 - acc: 0.7562 - val_loss: 5.8724 - val_acc: 0.2749
Epoch 288/500
1936/1936 [=======================

1936/1936 [==============================] - 0s 63us/step - loss: 0.6080 - acc: 0.7934 - val_loss: 5.8992 - val_acc: 0.2690
Epoch 341/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5961 - acc: 0.7872 - val_loss: 6.5912 - val_acc: 0.2778
Epoch 342/500
1936/1936 [==============================] - 0s 66us/step - loss: 0.6399 - acc: 0.7846 - val_loss: 6.1421 - val_acc: 0.2632
Epoch 343/500
1936/1936 [==============================] - 0s 61us/step - loss: 0.5916 - acc: 0.7877 - val_loss: 6.1664 - val_acc: 0.2719
Epoch 344/500
1936/1936 [==============================] - 0s 62us/step - loss: 0.5600 - acc: 0.7980 - val_loss: 6.1264 - val_acc: 0.2895
Epoch 345/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5920 - acc: 0.7929 - val_loss: 6.2076 - val_acc: 0.2749
Epoch 346/500
1936/1936 [==============================] - 0s 63us/step - loss: 0.6057 - acc: 0.7908 - val_loss: 5.9115 - val_acc: 0.2895
Epoch 347/500
1936/1936 [=======================

1936/1936 [==============================] - 0s 60us/step - loss: 0.5551 - acc: 0.8125 - val_loss: 6.1395 - val_acc: 0.2836
Epoch 400/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5214 - acc: 0.8223 - val_loss: 6.2961 - val_acc: 0.3070
Epoch 401/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5319 - acc: 0.8311 - val_loss: 6.1411 - val_acc: 0.3041
Epoch 402/500
1936/1936 [==============================] - 0s 62us/step - loss: 0.5465 - acc: 0.8239 - val_loss: 6.0464 - val_acc: 0.3129
Epoch 403/500
1936/1936 [==============================] - 0s 62us/step - loss: 0.5084 - acc: 0.8275 - val_loss: 5.9964 - val_acc: 0.3041
Epoch 404/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.4802 - acc: 0.8332 - val_loss: 5.9442 - val_acc: 0.2719
Epoch 405/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5358 - acc: 0.8146 - val_loss: 6.4138 - val_acc: 0.2953
Epoch 406/500
1936/1936 [=======================

1936/1936 [==============================] - 0s 59us/step - loss: 0.5459 - acc: 0.8151 - val_loss: 6.4734 - val_acc: 0.2807
Epoch 459/500
1936/1936 [==============================] - 0s 60us/step - loss: 0.5967 - acc: 0.8290 - val_loss: 6.2640 - val_acc: 0.2865
Epoch 460/500
1936/1936 [==============================] - 0s 63us/step - loss: 0.5792 - acc: 0.8125 - val_loss: 6.2973 - val_acc: 0.2924
Epoch 461/500
1936/1936 [==============================] - 0s 61us/step - loss: 0.4469 - acc: 0.8487 - val_loss: 6.3935 - val_acc: 0.2778
Epoch 462/500
1936/1936 [==============================] - 0s 63us/step - loss: 0.5229 - acc: 0.8419 - val_loss: 6.1145 - val_acc: 0.2982
Epoch 463/500
1936/1936 [==============================] - 0s 63us/step - loss: 0.4819 - acc: 0.8378 - val_loss: 6.5676 - val_acc: 0.2836
Epoch 464/500
1936/1936 [==============================] - 0s 61us/step - loss: 0.5113 - acc: 0.8316 - val_loss: 6.3955 - val_acc: 0.2924
Epoch 465/500
1936/1936 [=======================

# Interacting Materials

In [17]:
i_data = X[y == 1]
i_targ = df['h_target'][y == 1]

In [18]:
def gpu_mfcc_deep_net_i():
    # Create Model
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(114,)))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(31, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.summary()

    
    return model

In [ ]:
inter = KerasClassifier(build_fn=gpu_mfcc_deep_net_i, epochs=500, batch_size=128, validation_split=0.15)
history = inter.fit(i_data, i_targ)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               11500     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                6464      
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
__________

3213/3213 [==============================] - 0s 55us/step - loss: 1.9927 - acc: 0.3486 - val_loss: 2.4791 - val_acc: 0.2628
Epoch 49/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.9963 - acc: 0.3526 - val_loss: 2.4951 - val_acc: 0.2769
Epoch 50/500
3213/3213 [==============================] - 0s 55us/step - loss: 2.0044 - acc: 0.3480 - val_loss: 2.4809 - val_acc: 0.2787
Epoch 51/500
3213/3213 [==============================] - 0s 50us/step - loss: 1.9971 - acc: 0.3470 - val_loss: 2.4510 - val_acc: 0.2787
Epoch 52/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.9725 - acc: 0.3601 - val_loss: 2.4713 - val_acc: 0.2787
Epoch 53/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.9735 - acc: 0.3651 - val_loss: 2.4678 - val_acc: 0.2751
Epoch 54/500
3213/3213 [==============================] - 0s 55us/step - loss: 1.9703 - acc: 0.3582 - val_loss: 2.4232 - val_acc: 0.2716
Epoch 55/500
3213/3213 [==============================

Epoch 108/500
3213/3213 [==============================] - 0s 52us/step - loss: 1.6172 - acc: 0.4684 - val_loss: 2.6423 - val_acc: 0.3245
Epoch 109/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.6010 - acc: 0.4653 - val_loss: 2.6317 - val_acc: 0.3369
Epoch 110/500
3213/3213 [==============================] - 0s 49us/step - loss: 1.5937 - acc: 0.4712 - val_loss: 2.6797 - val_acc: 0.3404
Epoch 111/500
3213/3213 [==============================] - 0s 47us/step - loss: 1.5974 - acc: 0.4765 - val_loss: 2.6080 - val_acc: 0.3280
Epoch 112/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.5935 - acc: 0.4746 - val_loss: 2.5637 - val_acc: 0.3386
Epoch 113/500
3213/3213 [==============================] - 0s 52us/step - loss: 1.5836 - acc: 0.4805 - val_loss: 2.5697 - val_acc: 0.3351
Epoch 114/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.5716 - acc: 0.4877 - val_loss: 2.6538 - val_acc: 0.3316
Epoch 115/500
3213/3213 [=========

3213/3213 [==============================] - 0s 54us/step - loss: 1.3187 - acc: 0.5749 - val_loss: 2.7142 - val_acc: 0.3792
Epoch 168/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.3775 - acc: 0.5403 - val_loss: 2.7567 - val_acc: 0.3704
Epoch 169/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.3295 - acc: 0.5543 - val_loss: 2.7105 - val_acc: 0.3704
Epoch 170/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.3307 - acc: 0.5568 - val_loss: 2.7749 - val_acc: 0.3792
Epoch 171/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.3733 - acc: 0.5465 - val_loss: 2.7752 - val_acc: 0.3563
Epoch 172/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.3233 - acc: 0.5568 - val_loss: 2.8100 - val_acc: 0.3616
Epoch 173/500
3213/3213 [==============================] - 0s 56us/step - loss: 1.3217 - acc: 0.5503 - val_loss: 2.7163 - val_acc: 0.3757
Epoch 174/500
3213/3213 [=======================

3213/3213 [==============================] - 0s 54us/step - loss: 1.1846 - acc: 0.6032 - val_loss: 3.0001 - val_acc: 0.3845
Epoch 227/500
3213/3213 [==============================] - 0s 51us/step - loss: 1.1607 - acc: 0.6144 - val_loss: 3.0747 - val_acc: 0.3898
Epoch 228/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.1462 - acc: 0.6250 - val_loss: 3.1370 - val_acc: 0.3704
Epoch 229/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.1732 - acc: 0.6097 - val_loss: 3.0503 - val_acc: 0.3686
Epoch 230/500
3213/3213 [==============================] - 0s 52us/step - loss: 1.1648 - acc: 0.6119 - val_loss: 2.9337 - val_acc: 0.3968
Epoch 231/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.1359 - acc: 0.6222 - val_loss: 3.0519 - val_acc: 0.3757
Epoch 232/500
3213/3213 [==============================] - 0s 55us/step - loss: 1.0973 - acc: 0.6443 - val_loss: 3.0494 - val_acc: 0.3704
Epoch 233/500
3213/3213 [=======================

3213/3213 [==============================] - 0s 54us/step - loss: 1.0571 - acc: 0.6589 - val_loss: 3.2026 - val_acc: 0.3968
Epoch 286/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.0404 - acc: 0.6611 - val_loss: 3.1794 - val_acc: 0.3792
Epoch 287/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.0028 - acc: 0.6648 - val_loss: 3.2037 - val_acc: 0.3986
Epoch 288/500
3213/3213 [==============================] - 0s 55us/step - loss: 1.0127 - acc: 0.6632 - val_loss: 3.2456 - val_acc: 0.3651
Epoch 289/500
3213/3213 [==============================] - 0s 54us/step - loss: 1.0807 - acc: 0.6464 - val_loss: 3.2881 - val_acc: 0.3792
Epoch 290/500
3213/3213 [==============================] - 0s 53us/step - loss: 1.0440 - acc: 0.6495 - val_loss: 3.2568 - val_acc: 0.3862
Epoch 291/500
3213/3213 [==============================] - 0s 52us/step - loss: 1.0698 - acc: 0.6601 - val_loss: 3.1926 - val_acc: 0.3898
Epoch 292/500
3213/3213 [=======================

3213/3213 [==============================] - 0s 52us/step - loss: 0.9150 - acc: 0.6978 - val_loss: 3.3633 - val_acc: 0.3845
Epoch 345/500
3213/3213 [==============================] - 0s 50us/step - loss: 0.9502 - acc: 0.6928 - val_loss: 3.3707 - val_acc: 0.4004
Epoch 346/500
3213/3213 [==============================] - 0s 56us/step - loss: 0.9428 - acc: 0.6875 - val_loss: 3.3290 - val_acc: 0.3898
Epoch 347/500
3213/3213 [==============================] - 0s 56us/step - loss: 0.9726 - acc: 0.6772 - val_loss: 3.3161 - val_acc: 0.3898
Epoch 348/500
3213/3213 [==============================] - 0s 54us/step - loss: 0.9425 - acc: 0.6897 - val_loss: 3.3544 - val_acc: 0.4039
Epoch 349/500
3213/3213 [==============================] - 0s 55us/step - loss: 0.9236 - acc: 0.6934 - val_loss: 3.3305 - val_acc: 0.4004
Epoch 350/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.9257 - acc: 0.7000 - val_loss: 3.2804 - val_acc: 0.3862
Epoch 351/500
3213/3213 [=======================

3213/3213 [==============================] - 0s 54us/step - loss: 0.8925 - acc: 0.7006 - val_loss: 3.3246 - val_acc: 0.3898
Epoch 404/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.8318 - acc: 0.7289 - val_loss: 3.3584 - val_acc: 0.3915
Epoch 405/500
3213/3213 [==============================] - 0s 55us/step - loss: 0.8729 - acc: 0.7137 - val_loss: 3.2402 - val_acc: 0.4180
Epoch 406/500
3213/3213 [==============================] - 0s 54us/step - loss: 0.8463 - acc: 0.7274 - val_loss: 3.3580 - val_acc: 0.3915
Epoch 407/500
3213/3213 [==============================] - 0s 55us/step - loss: 0.8523 - acc: 0.7286 - val_loss: 3.3595 - val_acc: 0.4056
Epoch 408/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.8378 - acc: 0.7155 - val_loss: 3.4427 - val_acc: 0.3951
Epoch 409/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.8631 - acc: 0.7280 - val_loss: 3.4979 - val_acc: 0.3898
Epoch 410/500
3213/3213 [=======================

3213/3213 [==============================] - 0s 51us/step - loss: 0.7902 - acc: 0.7454 - val_loss: 3.5319 - val_acc: 0.3898
Epoch 463/500
3213/3213 [==============================] - 0s 52us/step - loss: 0.8255 - acc: 0.7333 - val_loss: 3.4439 - val_acc: 0.3968
Epoch 464/500
3213/3213 [==============================] - 0s 54us/step - loss: 0.8183 - acc: 0.7252 - val_loss: 3.4655 - val_acc: 0.3792
Epoch 465/500
3213/3213 [==============================] - 0s 54us/step - loss: 0.7669 - acc: 0.7460 - val_loss: 3.4738 - val_acc: 0.4039
Epoch 466/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.8160 - acc: 0.7392 - val_loss: 3.6183 - val_acc: 0.3933
Epoch 467/500
3213/3213 [==============================] - 0s 55us/step - loss: 0.7823 - acc: 0.7420 - val_loss: 3.5472 - val_acc: 0.3757
Epoch 468/500
3213/3213 [==============================] - 0s 53us/step - loss: 0.7510 - acc: 0.7532 - val_loss: 3.5491 - val_acc: 0.3810
Epoch 469/500
3213/3213 [=======================

# All Data

In [ ]:
data_iloc = 100
X_file = ps.preprocess_file(path_to_db + 'audio/' + dataset.iloc[data_iloc].filename,
                   blocksize=blocksize,
                   overlap=overlap)
X_file.head()

Epoch 1/1
2/2 [==============================] - 0s 200ms/step - loss: 1.2455
Epoch 1/1
2/2 [==============================] - 0s 183ms/step - loss: 1.3331


,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,64.457901,49.196175,40.925625,40.507687,41.320435,43.219643,34.976917,32.276566,35.604824,32.444164,...,-0.017353,0.023882,-0.018146,0.042829,0.004734,0.034921,0.007080,0.014718,0.004763,0.015325
1,90.546967,69.390068,50.230690,42.779140,46.435009,44.417530,44.154255,40.118462,39.088165,36.037605,...,-0.015659,0.030481,0.009894,-0.006443,-0.006340,-0.018149,0.013049,0.008803,0.003533,-0.014860
2,61.358559,70.692757,44.981976,46.210274,42.484043,40.296959,37.047440,33.908211,30.859825,33.457943,...,-0.040170,0.067244,-0.000768,0.017105,0.002783,0.013544,-0.003987,-0.014027,0.021521,0.020816
3,58.185257,64.353661,46.458714,61.875885,45.949047,41.506020,40.555374,44.917576,38.359520,39.579983,...,-0.001815,-0.018834,-0.003638,-0.015849,0.004869,0.007335,0.014557,0.015942,0.017861,0.007503


In [ ]:
pred_prob = clas.predict_proba(X_file)
pred = clas.predict(X_file)

In [ ]:
import numpy as np
np.average(pred_prob, axis=0)

array([0.61458333, 0.38541667])

In [ ]:
pred_prob = anim.predict_proba(X_file)
pred = anim.predict(X_file)

In [ ]:
print(pred_prob)

[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 1.486413e-38 2.062933e-30
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.0

In [ ]:
np.average(pred_prob, axis=0)

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.7160333e-39, 5.1573324e-31, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00], dtype=float32)

In [ ]:
from collections import Counter
b = Counter(pred)
print(b.most_common(1)[0][0])

20


In [ ]:
print(dataset.iloc[data_iloc])

filename      1-208757-B-2.wav
fold                         1
target                       2
category                   pig
esc10                    False
src_file                208757
take                         B
h_category                   0
Name: 100, dtype: object


# Test all

In [ ]:
df_fld = dataset[dataset['fold'] == 1]

In [ ]:
X_file = ps.preprocess_file(path_to_db + 'audio/' + df_fld.iloc[14].filename,
                   blocksize=blocksize,
                   overlap=overlap)
X_file


Epoch 1/1
2/2 [==============================] - 0s 195ms/step - loss: 1.2469
Epoch 1/1
2/2 [==============================] - 0s 213ms/step - loss: 1.3325


,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,62.097038,59.680424,30.253517,24.691685,24.575237,19.457026,15.262879,16.950579,10.196415,12.286251,...,-0.074145,0.033828,-0.046077,0.045997,0.048404,-0.009956,0.049822,-0.056967,0.05902,0.012229


In [ ]:
fld_pred = []
fld_h_pred = []
for i in range(0, len(df_fld)):
    print(str(i) + str(df_fld.iloc[i].category))
    X_file = ps.preprocess_file(path_to_db + 'audio/' + df_fld.iloc[i].filename,
                       blocksize=blocksize,
                       overlap=overlap)
    pred = clas.predict(X_file)
    prob = np.average(clas.predict_proba(X_file), axis=0)
    for i in range(0, len(pred)):
        if prob[0] > 0.5:
            fld_pred.append(anim.predict(X_file))
        else:
            fld_pred.append(inter.predict(X_file))

0dog
Epoch 1/1
2/2 [==============================] - 0s 193ms/step - loss: 1.2481
Epoch 1/1
2/2 [==============================] - 0s 217ms/step - loss: 1.3341
1chirping_birds
Epoch 1/1
2/2 [==============================] - 0s 220ms/step - loss: 1.2477
Epoch 1/1
2/2 [==============================] - 0s 228ms/step - loss: 1.3340
2vacuum_cleaner
Epoch 1/1
2/2 [==============================] - 0s 242ms/step - loss: 1.2488
Epoch 1/1
2/2 [==============================] - 0s 243ms/step - loss: 1.3315
3vacuum_cleaner
Epoch 1/1
2/2 [==============================] - 1s 260ms/step - loss: 1.2473
Epoch 1/1
2/2 [==============================] - 1s 265ms/step - loss: 1.3319
4thunderstorm
Epoch 1/1
2/2 [==============================] - 1s 287ms/step - loss: 1.2447
Epoch 1/1
2/2 [==============================] - 1s 277ms/step - loss: 1.3303
5thunderstorm
Epoch 1/1
2/2 [==============================] - 1s 285ms/step - loss: 1.2466
Epoch 1/1
2/2 [==============================] - 1s 290ms/ste

49cow
Epoch 1/1
2/2 [==============================] - 2s 951ms/step - loss: 1.2484
Epoch 1/1
2/2 [==============================] - 2s 883ms/step - loss: 1.3317
50water_drops
Epoch 1/1
2/2 [==============================] - 2s 851ms/step - loss: 1.2439
Epoch 1/1
2/2 [==============================] - 2s 811ms/step - loss: 1.3348
51brushing_teeth
Epoch 1/1
2/2 [==============================] - 2s 911ms/step - loss: 1.2461
Epoch 1/1
2/2 [==============================] - 2s 1s/step - loss: 1.3328
52brushing_teeth
Epoch 1/1
2/2 [==============================] - 2s 871ms/step - loss: 1.2463
Epoch 1/1
2/2 [==============================] - 2s 873ms/step - loss: 1.3334
53car_horn
Epoch 1/1
2/2 [==============================] - 3s 1s/step - loss: 1.2485
Epoch 1/1
2/2 [==============================] - 3s 1s/step - loss: 1.3335
54crackling_fire
Epoch 1/1
2/2 [==============================] - 3s 1s/step - loss: 1.2504
Epoch 1/1
2/2 [==============================] - 3s 1s/step - loss: 1.33

Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.3336
100pig
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.2485
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.3319
101pig
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.2505
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.3360
102pig
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.2467
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.3327
103pig
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.2458
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.3334
104crying_baby
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.2482
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.3331
105crying_baby
Epoch 1/1
2/2 [==============================] - 6s 3s/step - loss: 1.2504
Epoch 1/1
2/2 [===========================

In [ ]:
all_df.head()

In [ ]:
test_X = all_df.drop(['l_target', 'h_target'], axis=1)
test_y = all_df['l_target']

In [ ]:
import time
predictions = []
start_time = time.time()
top_pred_probs = []
pred_probs = []

pred = clas.predict(test_X)
prob = clas.predict_proba(test_X)
for i in range(0, len(pred)):
    if prob[0] > 0.5:
        pred[i] = anim.predict(test_X.iloc[i])
    else:
        pred[i] = inter.predict(test_X.iloc[i])
print(pred)
print(prob)

# for i in range(0,len(test_X)):
#     print(test_X.iloc[i])
#     pred = clas.predict(test_X.iloc[i])
#     top_pred_probs.append(clas.predict_proba(test_X.iloc[i]))
#     prob = np.average(top_pred_probs[-1], axis=0)
#     if prob[0] > 0.5:
#         pred = anim.predict(test_X.iloc[i])
#         b = Counter(pred)
#         predictions.append(b.most_common(1)[0][0])
#         pred_probs.append(anim.predict_proba(test_X.iloc[i]))
#     else:
#         pred = inter.predict(test_X.iloc[i])
#         b = Counter(pred)
#         predictions.append(b.most_common(1)[0][0])
#         pred_probs.append(inter.predict_proba(test_X.iloc[i]))
    
print("\tProcessing Time: " + str(time.time() - start_time))

In [ ]:
targs = np.vstack(full_targets)

In [ ]:
print(metrics.accuracy_score(full_targets, predictions))
cm = metrics.confusion_matrix(full_targets, predictions)
plt.figure(figsize=(20,20))
plot_confusion_matrix(cm, classes)
plt.show()

In [ ]:
import seaborn as sns

for i in range(0,20):
    prob = pred_probs[i]
    print(dataset.iloc[i])

    xs = np.unique(targets)
    ys = np.average(top_pred_probs[i], axis=0)

    plt.figure(figsize=(5,5))
    sns.barplot(xs, ys)
    
    xs = np.unique(a_targ)
    ys = np.average(prob, axis=0)
    if(len(ys)!=len(xs)):
        xs = np.unique(i_targ)

    plt.figure(figsize=(20,5))
    sns.barplot(xs, ys)
    plt.show()